# Trump generator

In [ ]:
import pandas

In [ ]:
tweets_data = pandas.read_csv("data.csv")

In [ ]:
tweets_data.head()

In [ ]:
tweets = tweets_data["Text"]

In [ ]:
tweets_str = tweets.str.cat(sep='\n')

In [ ]:
tweets_str[0:2000]

### Data cleaning

In [ ]:
import string

In [ ]:
def clean_data(text):
    #Split into tokens
    tokens = text.split()
    #Remove punctuation
    table = str.maketrans("","",string.punctuation)
    #Make tokens
    tokens = [t.translate(table)for t in tokens]
    #Remove non alpha
    tokens = [word for word in tokens if word.isalpha()]
    #Remove case
    tokens = [t.lower() for t in tokens]
    return tokens
    
    

In [ ]:
tweets_cleaned = clean_data(tweets_str)

In [ ]:
print(tweets_cleaned[:200])
print('Total Tokens: %d' % len(tweets_cleaned))
print('Unique Tokens: %d' % len(set(tweets_cleaned)))

In [ ]:
# organize into sequences of tokens
length = 50 + 1
sequences = list()
for i in range(length, len(tweets_cleaned)):
    # select sequence of tokens
    seq = tweets_cleaned[i-length:i]
    # convert into a line
    line = ' '.join(seq)
    # store
    sequences.append(line)
print('Total Sequences: %d' % len(sequences))

In [ ]:
def save_doc(lines, filename):
    data = '\n'.join(lines)
    file = open(filename, 'w')
    file.write(data)
    file.close()

In [ ]:
save_doc(sequences, "tweets-sequenced.txt")

## Train Language model

In [2]:
def load_doc(file):
    f = open(file,'r')
    txt = f.read()
    f.close()
    return txt

In [3]:
doc = load_doc("tweets-sequenced.txt")

In [4]:
lines = doc.split('\n')

In [ ]:
lines

### Encode sequences to integers

In [ ]:
import keras

In [5]:
from keras.preprocessing.text import Tokenizer

In [6]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)

In [7]:
vocab_size = len(tokenizer.word_index) + 1

In [ ]:
type(sequences)

In [8]:
import numpy as np

In [9]:
from keras.utils import np_utils

In [25]:
from keras.models import Sequential as seq
#from keras.layers import Embedding

In [20]:
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM
from keras.layers.core import Dense

In [ ]:
??np_utils.to_categorical

In [11]:
# separate into input and output
#sequences = array(sequences)
#sequences = [sequences]
sequences = np.asarray(sequences)
X, y = sequences[:,:-1], sequences[:,-1]
#y = np_utils.to_categorical(y, num_classes=vocab_size)
#y = np_utils.to_categorical(y, nb_classes=vocab_size)
seq_length = X.shape[1]

In [26]:
model = seq()
model.add(Embedding(vocab_size, 50, input_length=seq_length))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(100))
model.add(Dense(100, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))
#print(model.summary())

In [28]:
model.summary()

--------------------------------------------------------------------------------
Initial input shape: (None, 31367)
--------------------------------------------------------------------------------
Layer (name)                  Output Shape                  Param #             
--------------------------------------------------------------------------------
Embedding (Unnamed)           (None, 50, 50)                1568350             
LSTM (Unnamed)                (None, 50, 100)               60400               
LSTM (Unnamed)                (None, 100)                   80400               
Dense (Unnamed)               (None, 100)                   10100               
Dense (Unnamed)               (None, 31367)                 3168067             
--------------------------------------------------------------------------------
Total params: 4887317
--------------------------------------------------------------------------------


In [30]:
?? seq.compile

In [31]:
# compile model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')
# fit model
model.fit(X, y, batch_size=128, epochs=100)

Exception: Invalid objective: sparse_categorical_crossentropy